# Motor Speed vs Duty Cycle

Setup the environment:

In [2]:
import nest_asyncio, sys, os
nest_asyncio.apply()
sys.path.append(os.path.join(os.getenv('IOT_PROJECTS'), 'robot/code/rpi'))

In [ ]:
from asyncio_mqtt import Client
from struct import pack
import numpy as np
import asyncio, json, os

import stm32
from robot import *

print(f"plot @ http://{os.getenv('DNS_NAME')}.local:5006")

MQTT_BROKER = os.getenv("HOST_IP")
TOPIC_ROOT = "public/vis"

PARAM_DUTY1  = const(PARAM_RESERVED+0)   # motor1 duty cycle setpoint
PARAM_DUTY2  = const(PARAM_RESERVED+1)   # motor2 duty cycle setpoint

class DutyControl:

    def __init__(self):
        pass

    async def main(self):
        async with Client(MQTT_BROKER) as client, \
            Comm(self.state_listener) as robot:
            self.client = client
            await client.publish(f"{TOPIC_ROOT}/new", json.dumps({
                "columns": [ "duty [%]", "cpt" ],
                "rollover": 1000,
                "args": { "title": "Robot CPT vs Duty Cycle" },
            }))
            await robot.set(PARAM_FS, 5)
            await robot.start('duty_control')
            for duty in np.linspace(-100, 100, 50):
                await robot.set(PARAM_DUTY1, duty)
                await asyncio.sleep(1)
            for duty in np.linspace(100, -100, 50):
                await robot.set(PARAM_DUTY1, duty)
                await asyncio.sleep(1)

    async def state_listener(self, state):
        try:
            duty = state[STATE_DUTY1]
            cpt1 = state[STATE_CPT1]
            await self.client.publish(f"{TOPIC_ROOT}/bin", pack('!2f', duty, cpt1))
        except Exception as e:
            print("*****", e)


dc = DutyControl()
asyncio.run(dc.main())

Ah, there is a dead zone around zero. That's not surprising (and of course would be rather "worse" with a combustion engine). There is also a little hysteresis. This could pose problems with feedback controllers, especially balancing since that runs the motors around zero speed.

![](figures/cpt_vs_duty.png)

Let's zoom in:

In [5]:
import nest_asyncio, sys, os
nest_asyncio.apply()
sys.path.append(os.path.join(os.getenv('IOT_PROJECTS'), 'robot/code/rpi'))

from asyncio_mqtt import Client
from struct import pack
import numpy as np
import asyncio, json, os

import stm32
from robot import *

print(f"plot @ http://{os.getenv('DNS_NAME')}.local:5006")

MQTT_BROKER = os.getenv("HOST_IP")
TOPIC_ROOT = "public/vis"

PARAM_DUTY1  = const(PARAM_RESERVED+0)   # motor1 duty cycle setpoint
PARAM_DUTY2  = const(PARAM_RESERVED+1)   # motor2 duty cycle setpoint


class Control:

    def __init__(self, controller):
        self.controller = controller

    async def main(self):
        async with Client(MQTT_BROKER) as client, \
            Comm(self.state_listener) as robot:
            self.client = client
            await client.publish(f"{TOPIC_ROOT}/new", json.dumps({
                "columns": [ "duty [%]", "cpt" ],
                "rollover": 1000,
                "args": { "title": f"Robot {self.controller}" },
            }))
            await robot.set(PARAM_FS, 5)
            await robot.start(self.controller)
            range = 30
            N = 50
            for duty in np.linspace(-range, range, N):
                await robot.set(PARAM_DUTY1, duty)
                await asyncio.sleep(1)
            for duty in np.linspace(range, -range, N):
                await robot.set(PARAM_DUTY1, duty)
                await asyncio.sleep(1)

    async def state_listener(self, state):
        try:
            duty = PARAM[PARAM_DUTY1]
            cpt1 = state[STATE_CPT1]
            await self.client.publish(f"{TOPIC_ROOT}/bin", pack('!2f', duty, cpt1))
        except Exception as e:
            print("*****", e)


c = Control("duty_control")
asyncio.run(c.main())

plot @ http://pi4robot.local:5006
MCU: start Comm @ 1000000 baud
     


![](figures/duty_control.png)

Perhaps adding an offset to the duty input will improve the situation. `speed_control` does just that:

In [9]:
!cat $IOT_PROJECTS/robot/code/stm32/lib/speed_control.py

from controller import *
from param import PARAM

# Trivial "controller" that just turns on motors and sets pwm duty cycle.

PARAM_DUTY1  = const(2)   # motor1 duty cycle setpoint
PARAM_DUTY2  = const(3)   # motor2 duty cycle setpoint


class Control(Controller):
    
    @staticmethod
    def fix_duty(duty):
        if abs(duty) > 1:
            duty = duty+8.5 if duty>0 else duty-8.5
        return duty

    def update(self):
        # set motor duty cycle
        self.state[STATE_DUTY1] = self.fix_duty(PARAM[PARAM_DUTY1])
        self.state[STATE_DUTY2] = self.fix_duty(PARAM[PARAM_DUTY2])

In [ ]:
c = Control("speed_control")
asyncio.run(c.main())

![](figures/speed_control.png)

Not perfect, but much improved. Let's remember in case we run into problems later.